# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single kpoint and a small `Ecut` of 5 Hartree.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model, 5, kgrid=(1, 1, 1))

PlaneWaveBasis (Ecut=5.0, 1 kpoints)

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.122741e+02     1.566110e+00
 * time: 0.6679930686950684
     1     1.068124e+01     9.013168e-01
 * time: 1.9502429962158203
     2    -1.130942e+01     9.481453e-01
 * time: 2.5758049488067627
     3    -3.369645e+01     7.155944e-01
 * time: 3.488408088684082
     4    -4.717399e+01     5.061715e-01
 * time: 4.3732750415802
     5    -5.674253e+01     2.158875e-01
 * time: 5.246599912643433
     6    -5.970337e+01     1.484150e-01
 * time: 5.84299111366272
     7    -6.085820e+01     6.142199e-02
 * time: 6.425633907318115
     8    -6.130954e+01     5.701387e-02
 * time: 7.025686979293823
     9    -6.156155e+01     5.185429e-02
 * time: 7.618904113769531
    10    -6.177955e+01     2.541281e-02
 * time: 8.258821964263916
    11    -6.195519e+01     2.092514e-02
 * time: 8.830262899398804
    12    -6.204284e+01     1.732164e-02
 * time: 9.398788928985596
    13    -6.209774e+01     1.454891e-02
 * time: 10.00496792793274
    14 

In [5]:
scfres.energies

Energy breakdown:
    Kinetic             25.7671089
    AtomicLocal         -18.8557774
    AtomicNonlocal      14.8522698
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485409 
    Xc                  -19.3336835

    total               -62.261666457749
